IMPORT PACKAGES

In [1]:
import requests
import pandas as pd

PULL DATA FROM SOCRATA API

In [2]:
offset = 0
LIMIT = 1000000 #max rows which can be pulled from the api at a time
df_collection = [] #array which will hold multiple batches of data
while True:
    apiData = requests.get("https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$WHERE=created_date between '2017-01-01T00:00:00.000' and '2017-12-31T23:59:59.999' &$limit=" + str(LIMIT) + " &$offset=" + str(offset)).json()
    dfTemp = pd.DataFrame(data=apiData) #convert each batch of data into a dataframe so we can easily use the pandas concat method later
    df_collection.append(dfTemp)
    offset += LIMIT
    if (len(apiData)-LIMIT) >= 0: #check if the max limit was pulled from the api
        continue
    else: #near end of dataset, no need to send another request
        break
        
#concatenate all of the batches of data into one DataFrame
df = pd.concat(df_collection)

#test that all 2017 records are in the dataframe
#df['created_date'].min() = '2017-01-01T00:00:00.000'
#df['created_date'].max() = '2017-12-31T23:59:35.000'
print("#rows in dataframe: " + str(len(df)))

#rows in dataframe: 2445430


#Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

In [3]:
#filter out the top 10 complaint types
topTenComplaintTypes = df['complaint_type'].value_counts().head(10)
topTenComplaintTypes

Noise - Residential        230306
HEAT/HOT WATER             213520
Illegal Parking            146304
Blocked Driveway           136293
Street Condition            93270
Street Light Condition      84177
UNSANITARY CONDITION        79228
Noise - Street/Sidewalk     73115
Water System                65109
Noise                       60201
Name: complaint_type, dtype: int64

In [13]:
#filter the records for which the complaint type matches one found in 'topTenComplaintTypes'
dfTask1 = df.loc[df['complaint_type'].isin(topTenComplaintTypes.index)]
#filter the records to include only the 'borough' & 'complaint_type' (multiple for each borough), and the number of occurences for each complaint within that borough
dfTask1[['borough','complaint_type']].groupby(['borough','complaint_type'])['complaint_type'].count()

borough        complaint_type         
BRONX          Blocked Driveway           24600
               HEAT/HOT WATER             68717
               Illegal Parking            16154
               Noise                       3136
               Noise - Residential        57707
               Noise - Street/Sidewalk    14027
               Street Condition           11756
               Street Light Condition     18419
               UNSANITARY CONDITION       24526
               Water System               10186
BROOKLYN       Blocked Driveway           49385
               HEAT/HOT WATER             66984
               Illegal Parking            55458
               Noise                      15436
               Noise - Residential        67676
               Noise - Street/Sidewalk    21323
               Street Condition           25431
               Street Light Condition     22440
               UNSANITARY CONDITION       26656
               Water System               19826
M